# Hooks and Automation

Learn how to automate your Claude Code workflows using hooks - shell commands that execute automatically in response to events.

## What Are Hooks?

Hooks are shell commands that execute automatically when specific events occur in Claude Code:
- **Before/after tool calls** (file operations, bash commands, etc.)
- **When you submit prompts**
- **On session start/end**
- **Before git operations**

Think of hooks as automated guardrails and helpers that enhance your workflow!

## Why Use Hooks?

### Automated Quality Control
- Run linters before committing
- Execute tests automatically
- Validate code formatting
- Check for security issues

### Workflow Automation
- Auto-format code on save
- Update dependencies
- Generate documentation
- Sync with remote services

### Safety and Compliance
- Prevent accidental commits
- Enforce coding standards
- Block dangerous operations
- Log all changes

## Available Hook Types

### Tool Hooks
Execute before or after Claude uses tools:

| Hook | When It Runs |
|------|-------------|
| `before-tool` | Before any tool execution |
| `after-tool` | After any tool execution |
| `before-bash` | Before bash commands |
| `after-bash` | After bash commands |
| `before-write` | Before writing files |
| `after-write` | After writing files |
| `before-edit` | Before editing files |
| `after-edit` | After editing files |

### Session Hooks
Execute at session lifecycle events:

| Hook | When It Runs |
|------|-------------|
| `session-start` | When Claude Code starts |
| `session-end` | When Claude Code exits |
| `user-prompt-submit` | After you submit a message |

### Git Hooks
Execute during git operations:

| Hook | When It Runs |
|------|-------------|
| `pre-commit` | Before creating commits |
| `post-commit` | After commits are created |

## Configuring Hooks

### Configuration File Locations

**Project-level (recommended):**
```
.claude/settings.json
```
Applies to current project only.

**User-level:**
```
~/.claude/settings.json
```
Applies to all projects.

### Basic Hook Configuration

```json
{
  "hooks": {
    "session-start": "echo 'Claude Code started!'",
    "before-bash": "echo 'Running command...'",
    "after-write": "echo 'File written'"
  }
}
```

### View Current Hooks

```bash
/hooks          # View all configured hooks
/config hooks   # Edit hook configuration
```

## Hook Variables

Hooks have access to context variables depending on the hook type.

### Tool Hook Variables

| Variable | Description | Example |
|----------|-------------|--------|
| `$TOOL_NAME` | Name of the tool | `Write`, `Bash`, `Edit` |
| `$FILE_PATH` | File being operated on | `/path/to/file.py` |
| `$COMMAND` | Bash command (bash hooks) | `npm test` |

### Session Hook Variables

| Variable | Description |
|----------|-------------|
| `$SESSION_ID` | Current session ID |
| `$USER_PROMPT` | The prompt you submitted |
| `$CLAUDE_RESPONSE` | Claude's response (post-hooks) |

### Git Hook Variables

| Variable | Description |
|----------|-------------|
| `$COMMIT_MESSAGE` | The commit message |
| `$BRANCH_NAME` | Current git branch |
| `$STAGED_FILES` | List of staged files |

## Practical Hook Examples

### Example 1: Auto-Format Python Files

Automatically format Python files after editing:

```json
{
  "hooks": {
    "after-edit": "if [[ $FILE_PATH == *.py ]]; then black $FILE_PATH; fi"
  }
}
```

### Example 2: Run Tests Before Commits

Prevent commits if tests fail:

```json
{
  "hooks": {
    "pre-commit": "npm test || exit 1"
  }
}
```

The `exit 1` blocks the commit if tests fail!

### Example 3: Lint Check on File Write

Run linter after writing JavaScript files:

```json
{
  "hooks": {
    "after-write": "if [[ $FILE_PATH == *.js ]]; then eslint $FILE_PATH --fix; fi"
  }
}
```

### Example 4: Session Setup

Check for updates and install dependencies on start:

```json
{
  "hooks": {
    "session-start": "git pull && npm install"
  }
}
```

### Example 5: Security Check

Scan for secrets before committing:

```json
{
  "hooks": {
    "pre-commit": "gitleaks detect --staged || (echo 'Security issue detected!' && exit 1)"
  }
}
```

## Advanced Hook Patterns

### Multi-Command Hooks

Run multiple commands sequentially:

```json
{
  "hooks": {
    "pre-commit": "npm run lint && npm test && npm run build"
  }
}
```

### Conditional Hooks

Execute different commands based on conditions:

```json
{
  "hooks": {
    "after-write": "case $FILE_PATH in *.py) black $FILE_PATH;; *.js) prettier --write $FILE_PATH;; esac"
  }
}
```

### Hook Scripts

For complex logic, use external scripts:

```json
{
  "hooks": {
    "pre-commit": "./scripts/pre-commit-checks.sh"
  }
}
```

**scripts/pre-commit-checks.sh:**
```bash
#!/bin/bash
set -e

echo "Running pre-commit checks..."

# Run linter
echo "Linting..."
npm run lint

# Run tests
echo "Testing..."
npm test

# Check for TODOs
if git diff --cached | grep -i "TODO\|FIXME"; then
  echo "Warning: Found TODO/FIXME in staged changes"
  read -p "Continue anyway? (y/n) " -n 1 -r
  echo
  if [[ ! $REPLY =~ ^[Yy]$ ]]; then
    exit 1
  fi
fi

echo "All checks passed!"
```

## Blocking vs Non-Blocking Hooks

### Blocking Hooks (Use with Caution)

Block operations on failure:

```json
{
  "hooks": {
    "pre-commit": "npm test || exit 1"
  }
}
```

If the command exits with non-zero status, the operation is blocked.

### Non-Blocking Hooks (Recommended)

Show warnings but don't block:

```json
{
  "hooks": {
    "pre-commit": "npm test || echo 'Warning: Tests failed!'"
  }
}
```

### When to Block

**DO block for:**
- Critical security checks
- Breaking test failures
- Syntax errors
- Policy violations

**DON'T block for:**
- Linting warnings
- Non-critical tests
- Performance benchmarks
- Optional validations

## Language-Specific Hook Examples

### Python Project

```json
{
  "hooks": {
    "after-write": "if [[ $FILE_PATH == *.py ]]; then black $FILE_PATH && isort $FILE_PATH; fi",
    "pre-commit": "pytest && mypy . && flake8"
  }
}
```

### JavaScript/Node Project

```json
{
  "hooks": {
    "after-write": "if [[ $FILE_PATH == *.js || $FILE_PATH == *.jsx ]]; then prettier --write $FILE_PATH; fi",
    "pre-commit": "npm run lint && npm test"
  }
}
```

### TypeScript Project

```json
{
  "hooks": {
    "after-write": "if [[ $FILE_PATH == *.ts || $FILE_PATH == *.tsx ]]; then prettier --write $FILE_PATH; fi",
    "pre-commit": "tsc --noEmit && npm run lint && npm test"
  }
}
```

### Go Project

```json
{
  "hooks": {
    "after-write": "if [[ $FILE_PATH == *.go ]]; then gofmt -w $FILE_PATH; fi",
    "pre-commit": "go test ./... && go vet ./..."
  }
}
```

### Rust Project

```json
{
  "hooks": {
    "after-write": "if [[ $FILE_PATH == *.rs ]]; then rustfmt $FILE_PATH; fi",
    "pre-commit": "cargo test && cargo clippy"
  }
}
```

## Real-World Workflow Examples

### Full-Stack Development Workflow

```json
{
  "hooks": {
    "session-start": "git fetch && docker-compose up -d",
    "after-write": "case $FILE_PATH in *.py) black $FILE_PATH;; *.js|*.jsx) prettier --write $FILE_PATH;; esac",
    "pre-commit": "npm run lint && pytest && npm test",
    "session-end": "docker-compose down"
  }
}
```

### Data Science Workflow

```json
{
  "hooks": {
    "session-start": "jupyter notebook list || jupyter notebook --no-browser &",
    "after-write": "if [[ $FILE_PATH == *.ipynb ]]; then jupyter nbconvert --to notebook --execute $FILE_PATH --inplace; fi",
    "pre-commit": "pytest && jupyter nbconvert --clear-output --inplace notebooks/*.ipynb"
  }
}
```

### Microservices Workflow

```json
{
  "hooks": {
    "session-start": "make start-services",
    "pre-commit": "make test-all && make lint-all",
    "post-commit": "git push && make deploy-staging",
    "session-end": "make stop-services"
  }
}
```

## Debugging Hooks

### Enable Hook Logging

Add logging to see what hooks are doing:

```json
{
  "hooks": {
    "pre-commit": "echo 'Running pre-commit hook...' && npm test"
  }
}
```

### Test Hooks Manually

Run hook commands directly:

```bash
# Test what the hook would do
FILE_PATH="src/app.js" bash -c "if [[ $FILE_PATH == *.js ]]; then echo 'Would lint $FILE_PATH'; fi"
```

### Common Issues

**Hook not running:**
- Check syntax in settings.json
- Verify hook name is correct
- Look for typos in command

**Command fails silently:**
- Add explicit error output: `|| echo 'Failed!'`
- Check command exists: `which eslint`
- Test command manually first

**Hook blocks unexpectedly:**
- Check exit codes
- Make hook non-blocking: `|| true`
- Add conditional logic

## Performance Considerations

### Fast Hooks

```json
{
  "hooks": {
    "after-write": "echo 'File saved'"
  }
}
```
✅ Fast, no impact

### Slow Hooks

```json
{
  "hooks": {
    "after-write": "npm install && npm run build"
  }
}
```
❌ Slow, blocks workflow

### Optimize Hook Performance

**1. Run only when needed:**
```json
{
  "hooks": {
    "after-write": "if [[ $FILE_PATH == package.json ]]; then npm install; fi"
  }
}
```

**2. Use faster tools:**
```json
{
  "hooks": {
    "pre-commit": "eslint --cache && jest --onlyChanged"
  }
}
```

**3. Run in background:**
```json
{
  "hooks": {
    "session-start": "npm install > /dev/null 2>&1 &"
  }
}
```

## Best Practices

### 1. Keep Hooks Fast
```
✅ Quick linting: <1 second
✅ Targeted tests: <5 seconds
❌ Full build: >30 seconds
❌ All tests: >1 minute
```

### 2. Make Hooks Idempotent
```bash
# Good: Can run multiple times safely
"after-write": "black $FILE_PATH"

# Bad: Appends every time
"after-write": "echo '# Generated' >> $FILE_PATH"
```

### 3. Use Conditional Logic
```bash
# Only format Python files
"after-write": "if [[ $FILE_PATH == *.py ]]; then black $FILE_PATH; fi"
```

### 4. Provide Feedback
```bash
# Good: User knows what's happening
"pre-commit": "echo 'Running tests...' && npm test"

# Bad: Silent
"pre-commit": "npm test"
```

### 5. Handle Errors Gracefully
```bash
# Good: Clear error message
"pre-commit": "npm test || (echo 'Tests failed! Commit blocked.' && exit 1)"

# Bad: Cryptic failure
"pre-commit": "npm test"
```

### 6. Document Your Hooks
Add a README explaining what hooks do:
```markdown
## Hooks

This project uses the following hooks:
- `pre-commit`: Runs linter and tests before commits
- `after-write`: Auto-formats Python files
- `session-start`: Pulls latest changes and installs dependencies
```

## Security Considerations

### Be Careful with Hook Commands

❌ **Dangerous:**
```json
{
  "hooks": {
    "after-write": "curl $FILE_PATH | bash"
  }
}
```

✅ **Safe:**
```json
{
  "hooks": {
    "after-write": "black $FILE_PATH"
  }
}
```

### Validate Hook Sources
- Only use hooks from trusted sources
- Review hooks before enabling
- Avoid executing untrusted code

### Restrict Hook Permissions
```bash
# Make hook script executable by owner only
chmod 700 scripts/pre-commit-checks.sh
```

## Practical Exercises

### Exercise 1: Basic Auto-Format Hook (10 min)

1. Create a `.claude/settings.json` file
2. Add an `after-write` hook that formats Python files with black
3. Test by having Claude create or edit a Python file
4. Verify the file is auto-formatted

### Exercise 2: Pre-Commit Testing (15 min)

1. Add a `pre-commit` hook that runs tests
2. Create a test file with a failing test
3. Try to commit - it should be blocked
4. Fix the test and commit should succeed

### Exercise 3: Session Setup Hook (10 min)

1. Create a `session-start` hook that:
   - Prints a welcome message
   - Shows git status
   - Lists recent commits
2. Restart Claude Code to see it run

### Exercise 4: Multi-Language Formatting (20 min)

1. Create hooks that format different file types:
   - Python: black
   - JavaScript: prettier
   - JSON: jq
2. Test with files of each type
3. Verify correct formatter runs for each

### Exercise 5: Custom Hook Script (25 min)

1. Create a bash script for pre-commit checks
2. Include:
   - Linting
   - Testing
   - Security scan
   - TODO check
3. Configure pre-commit hook to run the script
4. Test all scenarios

## Troubleshooting Guide

### Hook Not Executing

**Check 1: Syntax**
```bash
# Validate JSON syntax
cat .claude/settings.json | jq
```

**Check 2: Hook Name**
```json
// ❌ Wrong
"precommit": "npm test"

// ✅ Correct
"pre-commit": "npm test"
```

**Check 3: Command Exists**
```bash
# Verify command is available
which black
which prettier
which npm
```

### Hook Fails Silently

**Add Logging:**
```json
{
  "hooks": {
    "pre-commit": "echo 'Starting tests...' && npm test 2>&1 | tee test.log"
  }
}
```

### Hook Too Slow

**Optimize:**
```json
{
  "hooks": {
    "pre-commit": "jest --onlyChanged --bail"
  }
}
```

**Or run in background:**
```json
{
  "hooks": {
    "session-start": "npm install &"
  }
}
```

## Key Takeaways

1. **Automate Quality:** Use hooks to enforce standards automatically
2. **Choose Right Hook Type:** Session, tool, or git hooks for different events
3. **Keep It Fast:** Optimize hooks to avoid slowing down workflow
4. **Provide Feedback:** Let users know what hooks are doing
5. **Handle Errors:** Graceful failures with clear messages
6. **Security First:** Only run trusted commands in hooks
7. **Document Everything:** Explain what hooks do and why

## Next Steps

Now that you understand hooks, continue to:
- **Notebook 04:** Skills Development - Create reusable capabilities
- **Notebook 05:** Slash Commands - Build custom commands
- **Notebook 06:** MCP Integrations - Connect external services

## Quick Reference

### Common Hook Patterns

```json
{
  "hooks": {
    // Auto-format on save
    "after-write": "prettier --write $FILE_PATH",
    
    // Run tests before commit
    "pre-commit": "npm test || exit 1",
    
    // Setup on start
    "session-start": "git pull && npm install",
    
    // Cleanup on exit
    "session-end": "docker-compose down"
  }
}
```

### Hook Command Shortcuts

| Command | Purpose |
|---------|--------|
| `/hooks` | View configured hooks |
| `/config hooks` | Edit hook settings |
| `/config` | View all settings |